# APIs

In [181]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Approach

### Problem Statement
Build a ML model to predict recommended sale price for a used car based on past sales trends

### Model Evaluation

The submitted model will be evaluated using Root Mean Squared Error (RMSE) on a hidden test set.

### Business Context
To be used by individuals to set selling price for their own cars while selling. This will enable them to make an informed decision and set a balanced price for the car, neither too high nor too low.

### Model

This is a regression task. Our task is to predict 'Price'.

Plan:
- Check missing values. Impute if required.
- Handle categorical variables.
- Scaling since we will start with Linear Regression as the Baseline.
- Univariate and Bivariate Analysis
- Check for outliers
- Correlation Analysis
- Feature Engineering
- Feature Selection
- Hyperparameter Tuning

# Data Exploration

## Loading training data

In [90]:
train_csv = pd.read_csv('data/train.csv')

In [91]:
train_csv.shape

(7478, 33)

In [92]:
train_csv.head()

,Id,Scraped_Time,Condition,Car Make,Model,Trim,Year,Kilometers,Body Type,Number of Seats,...,Neighborhood,Category,Subcategory,Interior Options,Exterior Options,Technology Options,Description_Score,Battery Capacity,Battery Range,Price
0,1,2025-02-07 01:58:30,Used,Toyota,Prius,Prius,2010,"20,000 - 29,999",Sedan,5,...,Abu Alanda,Cars and Bikes,Cars For Sale,"Airbags, Air Condition, Electric Seat Control,...","Panoramic Roof, Electrically Folding Mirrors, ...","Remote Start, Cruise Control, Radar, Navigatio...",فحص كامل,NaN,NaN,9350.0
1,2,2025-02-07 02:27:18,Used,Honda,Civic,Standard,2000,"+200,000",Sedan,4,...,Abu Alanda,Cars and Bikes,Cars For Sale,"Airbags, Center Lock, Air Condition, Alarm Sys...","Rim Wheels, Electric Mirrors",ABS Brakes,NaN,NaN,NaN,4200.0
2,3,2025-02-07 11:55:06,Used,Mitsubishi,Pajero,GLS,2015,"110,000 - 119,999",SUV,7,...,Sports City,Cars and Bikes,Cars For Sale,"Airbags, Steering Wheel Controls, Electric Win...","Spare Tyre, Rim Wheels, Electrically Folding M...","Media Screen, ABS Brakes, Rear Camera, Tractio...",NaN,NaN,NaN,8900.0
3,4,2025-02-07 13:25:57,Used,Nissan,Murano,SE,2005,"180,000 - 189,999",Sedan,5,...,Jabal Al Amera Rahma,Cars and Bikes,Cars For Sale,"Electric Seat Control, Air Condition, AUX / US...","Electrically Folding Mirrors, Electric Mirrors...","Traction Control, Cruise Control, Tyre Pressur...",فحص كامل,NaN,NaN,5500.0
4,5,2025-02-07 12:57:00,Used,Hyundai,H1,GL,2003,"140,000 - 149,999",Bus - Van,5,...,Al Souq,Cars and Bikes,Cars For Sale,"Steering Wheel Controls, Electric Windows, Cen...","Rear Sensors, Electric Mirrors, Daytime Runnin...","Bluetooth, Auto Lock System, Traction Control",NaN,NaN,NaN,5000.0


In [114]:
updated_col_names = ['Id', 'Scraped_Time', 'Condition', 'Car_Make', 'Model', 'Trim', 'Year',
       'Kilometers', 'Body_Type', 'Num_of_Seats', 'Fuel', 'Transmission',
       'Engine_Size_cc', 'Exterior_Color', 'Interior_Color',
       'Regional_Specs', 'Car_License', 'Insurance', 'Car_Customs',
       'Body_Condition', 'Paint', 'Payment_Method', 'City', 'Neighborhood',
       'Category', 'Subcategory', 'Interior_Options', 'Exterior_Options',
       'Technology_Options', 'Description_Score', 'Battery_Capacity',
       'Battery_Range', 'Price']

In [120]:
original_col_names = list(train_csv.columns)

In [122]:
cols_map = {}
for old, new in zip(original_col_names, updated_col_names):
    cols_map[old] = new.lower()
print(cols_map)   

{'Id': 'id', 'Scraped_Time': 'scraped_time', 'Condition': 'condition', 'Car Make': 'car_make', 'Model': 'model', 'Trim': 'trim', 'Year': 'year', 'Kilometers': 'kilometers', 'Body Type': 'body_type', 'Number of Seats': 'num_of_seats', 'Fuel': 'fuel', 'Transmission': 'transmission', 'Engine Size (cc)': 'engine_size_cc', 'Exterior Color': 'exterior_color', 'Interior Color': 'interior_color', 'Regional Specs': 'regional_specs', 'Car License': 'car_license', 'Insurance': 'insurance', 'Car Customs': 'car_customs', 'Body Condition': 'body_condition', 'Paint': 'paint', 'Payment Method': 'payment_method', 'City': 'city', 'Neighborhood': 'neighborhood', 'Category': 'category', 'Subcategory': 'subcategory', 'Interior Options': 'interior_options', 'Exterior Options': 'exterior_options', 'Technology Options': 'technology_options', 'Description_Score': 'description_score', 'Battery Capacity': 'battery_capacity', 'Battery Range': 'battery_range', 'Price': 'price'}


In [126]:
train_csv.rename(columns=cols_map, inplace=True)
train_csv.columns

Index(['id', 'scraped_time', 'condition', 'car_make', 'model', 'trim', 'year',
       'kilometers', 'body_type', 'num_of_seats', 'fuel', 'transmission',
       'engine_size_cc', 'exterior_color', 'interior_color', 'regional_specs',
       'car_license', 'insurance', 'car_customs', 'body_condition', 'paint',
       'payment_method', 'city', 'neighborhood', 'category', 'subcategory',
       'interior_options', 'exterior_options', 'technology_options',
       'description_score', 'battery_capacity', 'battery_range', 'price'],
      dtype='object')

## Check missing values

In [127]:
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7478 entries, 0 to 7477
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  7478 non-null   int64         
 1   scraped_time        7478 non-null   datetime64[ns]
 2   condition           7475 non-null   object        
 3   car_make            7475 non-null   object        
 4   model               7475 non-null   object        
 5   trim                6464 non-null   object        
 6   year                7474 non-null   object        
 7   kilometers          7471 non-null   object        
 8   body_type           7475 non-null   object        
 9   num_of_seats        7472 non-null   object        
 10  fuel                7471 non-null   object        
 11  transmission        7471 non-null   object        
 12  engine_size_cc      5933 non-null   object        
 13  exterior_color      7473 non-null   object      

In [128]:
null_pcnt_df = round(train_csv.isna().sum(axis=0)*100/len(train_csv),2)

Features with greater than 10% null values need to be studied in detail.

In [129]:
null_pcnt_df[null_pcnt_df>10]

trim                 13.56
engine_size_cc       20.66
description_score    59.59
battery_capacity     83.38
battery_range        83.38
dtype: float64

## Data Cleaning and transformation

In [130]:
train_csv.columns

Index(['id', 'scraped_time', 'condition', 'car_make', 'model', 'trim', 'year',
       'kilometers', 'body_type', 'num_of_seats', 'fuel', 'transmission',
       'engine_size_cc', 'exterior_color', 'interior_color', 'regional_specs',
       'car_license', 'insurance', 'car_customs', 'body_condition', 'paint',
       'payment_method', 'city', 'neighborhood', 'category', 'subcategory',
       'interior_options', 'exterior_options', 'technology_options',
       'description_score', 'battery_capacity', 'battery_range', 'price'],
      dtype='object')

In [159]:
len(train_csv)

7478

### scraped_time

Changing the data type to datetime

In [131]:
train_csv['scraped_time'] = pd.to_datetime(train_csv['scraped_time'])

In [133]:
print(train_csv.scraped_time.dtype)

datetime64[ns]


In [134]:
train_csv.scraped_time.describe()

count                             7478
mean     2025-02-07 07:29:41.780422656
min                2025-02-06 20:16:48
25%      2025-02-06 23:46:47.249999872
50%                2025-02-07 02:24:16
75%         2025-02-07 12:07:14.500000
max                2025-02-10 00:11:05
Name: scraped_time, dtype: object

This feature provides the time when the data was scraped i.e. collected. The date range is Feb 6th to Feb 10th, 2025. This date will not have an impact on our target.

If instead, we had the date of transaction, it might had some significance. We could have extracted day of the week, month, day of the month, etc to test for seasonality.

### condition

In [135]:
train_csv.condition.describe()

count     7475
unique       2
top       Used
freq      7104
Name: condition, dtype: object

In [136]:
train_csv.condition.value_counts()

condition
Used    7104
New      371
Name: count, dtype: int64

3 rows have data missing.

We can either use Label or One Hot Encoder for this feature.

In [137]:
df = train_csv[['condition', 'price']]
avg_price_by_condn = df.groupby('condition').agg({
    'price': ['mean', 'median']})
avg_price_by_condn

price         
                   mean   median
condition                       
New        20559.026954  18800.0
Used       11838.153012   8000.0

As expected, the average or median Price for New cars is higher than Used cars, almost double in value.

### car_make

In [138]:
train_csv.car_make.describe()

count        7475
unique         74
top       Hyundai
freq         1289
Name: car_make, dtype: object

There are 74 unique car makers and 3 rows with missing values. 

Since there are many columns with missing values, I am thinking KNN imputer or a simple median imputer may help.

In [154]:
top_20_car_makers = train_csv.car_make.value_counts().reset_index()[:20]
top_20_car_makers

,car_make,count
0,Hyundai,1289
1,Kia,965
2,Toyota,811
3,Mercedes Benz,518
4,Mitsubishi,428
5,Volkswagen,371
6,Ford,325
7,Nissan,281
8,Honda,270
9,Chevrolet,218


In [155]:
print(f'Top 20 car makers contribute to {round(top_20_car_makers['count'].sum()*100/train_csv['car_make'].count(), 2)} % of training data')

Top 20 car makers contribute to 89.57 % of training data


### model

In [162]:
train_csv['model'].describe()

count       7475
unique       512
top       Avante
freq         325
Name: model, dtype: object

In [161]:
train_csv['model'].value_counts()

model
Avante     325
Lancer     226
E-Class    207
Accent     195
Prius      187
          ... 
MKX          1
Montero      1
MG 6         1
Macan        1
Charade      1
Name: count, Length: 512, dtype: int64

Since there are so many models, some models will be rare. Also, the test data may have a model which is not present in the training data. To handle such cases, I am thinking we need to consider model by car_maker, and while encoding label less common models as unknown/other.

### Trim

In [163]:
train_csv['trim'].describe()

count         6464
unique         546
top       Standard
freq          1330
Name: trim, dtype: object

This feature has a lot of missing values.

In [164]:
train_csv['trim'].value_counts()

trim
Standard                          1330
Limited                            349
SE                                 238
EX                                 210
E 200                              163
                                  ... 
i3 eDrive40L Yaoye Sports Suit       1
WRX                                  1
Latitude LUX                         1
SV2                                  1
PHEV                                 1
Name: count, Length: 546, dtype: int64

In [165]:
train_csv['trim'].value_counts().reset_index()[:10]

,trim,count
0,Standard,1330
1,Limited,349
2,SE,238
3,EX,210
4,E 200,163
5,GL,130
6,Prius,121
7,GLX,117
8,GLS,113
9,S,109


# Splitting Training data into Train and Test sets

In [172]:
y = train_csv['price']
X = train_csv.drop(columns=['price', 'scraped_time', 'id'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=True)

In [173]:
print(X_train.shape, y_train.shape,'\n', X_test.shape, y_test.shape)

(5608, 30) (5608,) 
 (1870, 30) (1870,)


# Baseline Model

Random Forest

In [174]:
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns
num_cols = X_train.select_dtypes(include=['number']).columns
print(f'Category Columns:\n{cat_cols}')
print(f'Numerical Columns:\n{num_cols}')

Category Columns:
Index(['condition', 'car_make', 'model', 'trim', 'year', 'kilometers',
       'body_type', 'num_of_seats', 'fuel', 'transmission', 'engine_size_cc',
       'exterior_color', 'interior_color', 'regional_specs', 'car_license',
       'insurance', 'car_customs', 'body_condition', 'paint', 'payment_method',
       'city', 'neighborhood', 'category', 'subcategory', 'interior_options',
       'exterior_options', 'technology_options', 'description_score',
       'battery_capacity', 'battery_range'],
      dtype='object')
Numerical Columns:
Index([], dtype='object')


In [182]:
base_pipe = make_pipeline(OneHotEncoder(), LinearRegression())

In [170]:
# param_grid = {'randomforestregressor__max_features' : ['sqrt', None]}
# grid = GridSearchCV(base_pipe, param_grid=param_grid, scoring='neg_root_mean_squared_error', n_jobs=-1)
# grid.fit(X_train, y_train)

In [186]:
# cv = cross_val_score(base_pipe, X_train, y_train, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=5)

In [86]:
ohe = grid.best_estimator_.named_steps['onehotencoder']
X_train_encoded = ohe.transform(X_train)
print(X_train_encoded.shape)

(5608, 10711)
